<a href="https://colab.research.google.com/github/mohansameer1983/ComputerVisionProjects/blob/main/Trash_Multilabel_Classification_CV_Neptune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Uncomment only for first run
!pip install neptune-client neptune-tensorflow-keras tensorflow

     |████████████████████████████████| 315 kB 3.9 MB/s 
     |████████████████████████████████| 829 kB 64.5 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 181 kB 66.4 MB/s 
     |████████████████████████████████| 132 kB 66.3 MB/s 
     |████████████████████████████████| 79 kB 9.1 MB/s 
     |████████████████████████████████| 8.6 MB 32.5 MB/s 
     |████████████████████████████████| 138 kB 59.7 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 127 kB 58.2 MB/s 
     |████████████████████████████████| 462 kB 55.1 MB/s 
     |████████████████████████████████| 130 kB 62.2 MB/s 
     |████████████████████████████████| 67 kB 3.1 MB/s 
     |████████████████████████████████| 64 kB 3.0 MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.15.1-py2.py3-none-any.whl size=564805 sha256=f49629141af30e30109ef168c4a1a94f611745d7ebadb0a892f5df87678eeaf9
  Stored in directory: /roo

In [2]:

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime

import neptune.new as neptune
from neptune.new.integrations.tensorflow_keras import NeptuneCallback

import cv2
from google.colab.patches import cv2_imshow      # To show images using cv2 module
from pycocotools.coco import COCO

import pathlib
import os

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Flatten

from keras.preprocessing.image import ImageDataGenerator

import tensorflow_hub as hub

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

TF version: 2.8.0
Hub version: 0.12.0
GPU is available


In [3]:
# Mount Google drive so dataset can be accessed
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')
from prepare_dataset import convert_annotation_json_to_dataframe

In [5]:
run = neptune.init(
    project="mohansameer/Trash-Classification-Deep-Learning",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI5ZjI5MDY4Yi0zMWVlLTQ3MmMtYjE4NS1iNDRjNGM0OTg4ZDkifQ==",
) 

https://app.neptune.ai/mohansameer/Trash-Classification-Deep-Learning/e/TRAS-22
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


# Model Selection

In [6]:
  # Select base model  for transfer learning
  model_name = "mobilenet_v2_140_224" # @param [ 'efficientnet_v2_50', 'resnet_v1_50', 'resnet_v1_101', 'resnet_v1_152', 'resnet_v2_50', 'resnet_v2_101', 'resnet_v2_152',  'mobilenet_v2_100_224', 'mobilenet_v2_130_224', 'mobilenet_v2_140_224', 'mobilenet_v3_small_100_224', 'mobilenet_v3_small_075_224', 'mobilenet_v3_large_100_224', 'mobilenet_v3_large_075_224']

  model_handle_map = {
    "efficientnet_v2_50": "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet1k_b0/feature_vector/2",   
    "resnet_v1_50": "https://tfhub.dev/google/imagenet/resnet_v1_50/feature-vector/4",
    "resnet_v1_101": "https://tfhub.dev/google/imagenet/resnet_v1_101/feature-vector/4",
    "resnet_v1_152": "https://tfhub.dev/google/imagenet/resnet_v1_152/feature-vector/4",
    "resnet_v2_50": "https://tfhub.dev/google/imagenet/resnet_v2_50/feature-vector/4",
    "resnet_v2_101": "https://tfhub.dev/google/imagenet/resnet_v2_101/feature_vector/5",
    "resnet_v2_152": "https://tfhub.dev/google/imagenet/resnet_v2_152/feature-vector/4",
    "nasnet_large": "https://tfhub.dev/google/imagenet/nasnet_large/feature_vector/4",
    "nasnet_mobile": "https://tfhub.dev/google/imagenet/nasnet_mobile/feature_vector/4",
    "pnasnet_large": "https://tfhub.dev/google/imagenet/pnasnet_large/feature_vector/4",
    "mobilenet_v2_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/4",
    "mobilenet_v2_130_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/feature_vector/4",
    "mobilenet_v2_140_224": "https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/4",
    "mobilenet_v3_small_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/feature_vector/5",
    "mobilenet_v3_small_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_small_075_224/feature_vector/5",
    "mobilenet_v3_large_100_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_100_224/feature_vector/5",
    "mobilenet_v3_large_075_224": "https://tfhub.dev/google/imagenet/mobilenet_v3_large_075_224/feature_vector/5",
  }

  PIXELS = 224#@param {type:"integer"}
  BATCH_SIZE = 16#@param {type:"integer"}

  model_handle = model_handle_map.get(model_name)

  print(f"Selected model: {model_name} : {model_handle}")

  IMAGE_SIZE = (PIXELS, PIXELS)
  print(f"Input size {IMAGE_SIZE}")


Selected model: mobilenet_v2_140_224 : https://tfhub.dev/google/imagenet/mobilenet_v2_140_224/feature_vector/4
Input size (224, 224)


In [7]:
#Log to NeptuneAI
run["model/transferlearning/model_name"] = model_name
#run["model/transferlearning/model_url"] = model_handle
run["model/transferlearning/model_batch_size"] = BATCH_SIZE
run["model/parameters/image_size"] = IMAGE_SIZE

# Data Preparation

In [8]:
image_dataframe, grouped_image_dataframe, labels_dict = convert_annotation_json_to_dataframe('/content/drive/MyDrive/Colab Notebooks/data/annotations.json')

grouped_image_dataframe['file_name'] = grouped_image_dataframe['file_name'].str.replace('/','_')
grouped_image_dataframe

2022-03-15 04:14:38,505 - Number of Categories: 60 ; Categories: [{0: 'Aluminium foil', 1: 'Battery', 2: 'Aluminium blister pack', 3: 'Carded blister pack', 4: 'Other plastic bottle', 5: 'Clear plastic bottle', 6: 'Glass bottle', 7: 'Plastic bottle cap', 8: 'Metal bottle cap', 9: 'Broken glass', 10: 'Food Can', 11: 'Aerosol', 12: 'Drink can', 13: 'Toilet tube', 14: 'Other carton', 15: 'Egg carton', 16: 'Drink carton', 17: 'Corrugated carton', 18: 'Meal carton', 19: 'Pizza box', 20: 'Paper cup', 21: 'Disposable plastic cup', 22: 'Foam cup', 23: 'Glass cup', 24: 'Other plastic cup', 25: 'Food waste', 26: 'Glass jar', 27: 'Plastic lid', 28: 'Metal lid', 29: 'Other plastic', 30: 'Magazine paper', 31: 'Tissues', 32: 'Wrapping paper', 33: 'Normal paper', 34: 'Paper bag', 35: 'Plastified paper bag', 36: 'Plastic film', 37: 'Six pack rings', 38: 'Garbage bag', 39: 'Other plastic wrapper', 40: 'Single-use carrier bag', 41: 'Polypropylene bag', 42: 'Crisp packet', 43: 'Spread tub', 44: 'Tupperwa

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4784 entries, 0 to 4783
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   annotation_id  4784 non-null   int64 
 1   image_id       4784 non-null   int64 
 2   category_id    4784 non-null   int64 
 3   file_name      4784 non-null   object
 4   category_name  4784 non-null   object
dtypes: int64(3), object(2)
memory usage: 224.2+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 3123 entries, 0 to 4783
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   annotation_id  3123 non-null   int64 
 1   image_id       3123 non-null   int64 
 2   category_id    3123 non-null   int64 
 3   file_name      3123 non-null   object
 4   category_name  3123 non-null   object
dtypes: int64(3), object(2)
memory usage: 146.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data col

,file_name,labels
0,batch_1_000000.jpg,[Clear plastic bottle]
1,batch_1_000001.jpg,"[Clear plastic bottle, Plastic bottle cap]"
2,batch_1_000003.jpg,[Paper cup]
3,batch_1_000004.jpg,[Drink can]
4,batch_1_000005.jpg,[Clear plastic bottle]
...,...,...
1495,batch_9_000095.jpg,"[Drink carton, Plastic straw, Unlabeled litter..."
1496,batch_9_000096.jpg,[Plastic straw]
1497,batch_9_000097.jpg,"[Glass bottle, Metal bottle cap]"
1498,batch_9_000098.jpg,"[Drink carton, Plastic bottle cap]"


In [9]:
def load_data(save_dir="./"):

  train_data_path = '/content/drive/MyDrive/Colab Notebooks/data/trash_grouped/train_preprocessed/'
  test_data_path = '/content/drive/MyDrive/Colab Notebooks/data/trash_grouped/test_preprocessed/'

  datagen_kwargs = dict(rescale=1./255, validation_split=0.2)


  dataflow_kwargs = dict(target_size=(PIXELS, PIXELS), 
                        batch_size=BATCH_SIZE,
                        class_mode="categorical",
                        x_col='file_name',
                        y_col='labels')

  train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      horizontal_flip=True, vertical_flip=True, rotation_range=90,
      **datagen_kwargs)
  train_generator = train_datagen.flow_from_dataframe(
      grouped_image_dataframe,
      directory=train_data_path,
      subset='training',
      **dataflow_kwargs
  )


  val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      **datagen_kwargs)
  val_generator = val_datagen.flow_from_dataframe(
      grouped_image_dataframe,
      directory=train_data_path,
      subset='validation',
      **dataflow_kwargs
  )
  return val_generator, train_generator

VAL_generator, TRAIN_generator = load_data()

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 285 invalid image filename(s) in x_col="file_name". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 972 validated image filenames belonging to 59 classes.
Found 243 validated image filenames belonging to 59 classes.


In [10]:
#from sklearn.preprocessing import MultiLabelBinarizer

# Create the Scikit-learn MultiLabelBinarizer
'''mlb = MultiLabelBinarizer()
mlb.fit(grouped_image_dataframe['labels'].values.tolist())

# The helper function
def multilabel_flow_from_dataframe(data_generator, mlb):
    for x, y in data_generator:
        assert isinstance(mlb, MultiLabelBinarizer), "MultiLabelBinarizer is required."
        indices = y.astype(int).tolist()

        print("indices:", grouped_image_dataframe.iloc[indices]['labels'].values.tolist())
        y_multi = mlb.transform(
             grouped_image_dataframe.iloc[indices]['labels'].values.tolist()
        )
        print("y_multi:", y_multi)
        yield x, y_multi       
        
        # Use the helper function to create a multi-label image generator
train_multilabel_generator = multilabel_flow_from_dataframe(
    TRAIN_generator, 
    mlb
)

val_multilabel_generator = multilabel_flow_from_dataframe(
    VAL_generator, 
    mlb
)
        '''

'mlb = MultiLabelBinarizer()\nmlb.fit(grouped_image_dataframe[\'labels\'].values.tolist())\n\n# The helper function\ndef multilabel_flow_from_dataframe(data_generator, mlb):\n    for x, y in data_generator:\n        assert isinstance(mlb, MultiLabelBinarizer), "MultiLabelBinarizer is required."\n        indices = y.astype(int).tolist()\n\n        print("indices:", grouped_image_dataframe.iloc[indices][\'labels\'].values.tolist())\n        y_multi = mlb.transform(\n             grouped_image_dataframe.iloc[indices][\'labels\'].values.tolist()\n        )\n        print("y_multi:", y_multi)\n        yield x, y_multi       \n        \n        # Use the helper function to create a multi-label image generator\ntrain_multilabel_generator = multilabel_flow_from_dataframe(\n    TRAIN_generator, \n    mlb\n)\n\nval_multilabel_generator = multilabel_flow_from_dataframe(\n    VAL_generator, \n    mlb\n)\n        '

# Modelling

## Build and Train Model


In [11]:
def build_model(trainable=False, params={}):

    #imports the mobilenet model and discards the last 1000 neuron layer.
   base_model=keras.applications.MobileNetV2(weights='imagenet',
                                             include_top=False,
                                            input_shape=IMAGE_SIZE+(3,))

   base_model.trainable = False                                      

   inputs = keras.Input(shape=IMAGE_SIZE+(3,))
   x = base_model(inputs, training=False)
   x = keras.layers.GlobalAveragePooling2D()(x)
   x=Dense(params['dense_units_1'],activation=params['dense_layer_activation_1'])(x)
   x=Dropout(params['dropout_1'])(x)
   x=Dense(params['dense_units_2'],activation=params['dense_layer_activation_2'])(x)
   x=Dropout(params['dropout_2'])(x)
   x = Flatten()(x)
   x=Dense(params['dense_units_2'],activation=params['dense_layer_activation_2'])(x)
   outputs = keras.layers.Dense(59, activation=params['final_layer_activation'])(x)
   model = keras.Model(inputs, outputs)

   model.summary()
   return model

params = {
    "dense_units_1": 512,
    "dense_layer_activation_1": "relu",
    "dropout_1": 0.3,    
    "dense_units_2": 128,
    "dense_layer_activation_2": "relu",
    "dropout_2": 0.1,
    "final_layer_activation": "sigmoid",
    "learning_rate": 1e-2,
    "n_epochs": 50,    
    "regularizer_rate":0.005
}
run["model/parameters"] = params
model = build_model(params=params)

9420800/9406464 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               655872    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                

In [12]:
from keras import backend
 
# calculate fbeta score for multi-class/label classification
def fbeta(y_true, y_pred, beta=2):
	# clip predictions
	y_pred = backend.clip(y_pred, 0, 1)
	# calculate elements
	tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
	fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
	fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)
	# calculate precision
	p = tp / (tp + fp + backend.epsilon())
	# calculate recall
	r = tp / (tp + fn + backend.epsilon())
	# calculate fbeta, averaged across each class
	bb = beta ** 2
	fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
	return fbeta_score

In [13]:
def compile_model(model):
  model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=params['learning_rate']), 
  loss=keras.losses.BinaryCrossentropy(),
  metrics=[fbeta])
  return model

model = compile_model(model)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [14]:
# Add Callbacks

# Model Checkpoint
tl_checkpoint_1 = ModelCheckpoint(filepath = '/content/drive/MyDrive/Colab Notebooks/Models/garbageclassification_model_checkpoint/resnet_best_weights.hdf5', save_best_only = True, verbose = 0)

# Early Stopping Checkpoint
EarlyStoppingCallback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=10)

In [ ]:
def train_model(model, num_epochs):
    steps_per_epoch = TRAIN_generator.samples // TRAIN_generator.batch_size
    validation_steps = VAL_generator.samples // VAL_generator.batch_size

    history = model.fit(
      TRAIN_generator,
      epochs=num_epochs, steps_per_epoch=steps_per_epoch,
      validation_data = VAL_generator,
      validation_steps = validation_steps,
      callbacks=[NeptuneCallback(run=run),EarlyStoppingCallback]).history
    
    return model, history

model, history = train_model(model, num_epochs=params['n_epochs'])

Epoch 1/50
60/60 [==============================] - ETA: 0s - loss: 0.1307 - fbeta: 0.0046WARNING:tensorflow:Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: loss,fbeta


2022-03-15 04:38:54,361 - Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: loss,fbeta


60/60 [==============================] - 114s 2s/step - loss: 0.1307 - fbeta: 0.0046
Epoch 2/50
60/60 [==============================] - ETA: 0s - loss: 0.1287 - fbeta: 0.0025WARNING:tensorflow:Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: loss,fbeta


2022-03-15 04:41:15,683 - Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: loss,fbeta


60/60 [==============================] - 114s 2s/step - loss: 0.1287 - fbeta: 0.0025
Epoch 3/50
60/60 [==============================] - ETA: 0s - loss: 0.1255 - fbeta: 0.0068WARNING:tensorflow:Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: loss,fbeta


...

60/60 [==============================] - 114s 2s/step - loss: 0.1255 - fbeta: 0.0068
Epoch 4/50
60/60 [==============================] - ETA: 0s - loss: 0.1252 - fbeta: 0.0047WARNING:tensorflow:Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: loss,fbeta


2022-03-15 04:45:06,385 - Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: loss,fbeta


60/60 [==============================] - 114s 2s/step - loss: 0.1252 - fbeta: 0.0047
Epoch 5/50
60/60 [==============================] - ETA: 0s - loss: 0.1231 - fbeta: 0.0076WARNING:tensorflow:Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: loss,fbeta


2022-03-15 04:47:28,387 - Early stopping conditioned on metric `accuracy` which is not available. Available metrics are: loss,fbeta


60/60 [==============================] - 114s 2s/step - loss: 0.1231 - fbeta: 0.0076
Epoch 6/50
 8/60 [===>..........................] - ETA: 1:33 - loss: 0.1237 - fbeta: 0.0000e+00

In [ ]:
plt.figure()
plt.ylabel("fbeta (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,2])
plt.plot(history["fbeta"])
plt.plot(history["val_fbeta"])

plt.figure()
plt.ylabel("Accuracy (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(history["accuracy"])
plt.plot(history["val_accuracy"])

In [ ]:
# define filename
filename = '/content/drive/MyDrive/Colab Notebooks/data/trash/Aerosol/batch_1_000054.jpg'

img = tf.keras.preprocessing.image.load_img(
    filename, target_size=(224, 224)
)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
print(predictions)
score = tf.nn.sigmoid(predictions[0])
print(score)
print(list(class_names))
print( "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(list(class_names)[np.argmax(score)], 100 * np.max(score)) )

In [ ]:
# Generate predictions
model.load_weights('/content/drive/MyDrive/Colab Notebooks/Models/garbageclassification_model_checkpoint/resnet_best_weights.hdf5') # initialize the best trained weights
preds = model.predict(test_ds)
pred_classes = np.argmax(preds, axis = 1)

OSError: ignored

# Save Model

In [ ]:
saved_model_path = "/content/drive/MyDrive/Colab Notebooks/Models/saved_garbageclassification_model"
tf.saved_model.save(model, saved_model_path)

# Fine Tuning